In [5]:
from transformers import pipeline
classifier=pipeline("sentiment-analysis")
raw_inputs=["I've been waiting for a HuggingFace course my whole life.","I hate this so much!",]
result=classifier(raw_inputs)
result

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9598048329353333},
 {'label': 'NEGATIVE', 'score': 0.9994558691978455}]

In [4]:
#preprocessing with the tokenizer
#converting text input to number
from transformers import AutoTokenizer
tokenizer=AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [8]:
inputs=tokenizer(raw_inputs,
                 padding=True,
                 truncation=True,# This ensures that sequences longer than the maximum allowed length are truncated to fit within that limit.
                 return_tensors="tf")
inputs

{'input_ids': <tf.Tensor: shape=(2, 16), dtype=int32, numpy=
array([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662,
        12172,  2607,  2026,  2878,  2166,  1012,   102],
       [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,
            0,     0,     0,     0,     0,     0,     0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2, 16), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>}

In [12]:
#loading our pretrained model
from transformers import TFAutoModel
model=TFAutoModel.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['pre_classifier.bias', 'classifier.bias', 'classifier.weight', 'pre_classifier.weight']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [13]:
outputs=model(inputs)
print(outputs.last_hidden_state.shape)

(2, 16, 768)


Batch Size=2,
Sequence length=16,
Hidden size=768

In [16]:
#If we want our model to classify sentences as positive or negative we use
from transformers import TFAutoModelForSequenceClassification
model=TFAutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
outputs=model(inputs)

All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


In [17]:
outputs.logits.shape# since we have only two sentences and two labels its shape is 2x2

TensorShape([2, 2])

In [18]:
#postprocessing the output
print(outputs.logits)#logits, the raw, unnormalized scores outputted by the last layer of the model

tf.Tensor(
[[-1.5606961  1.6122813]
 [ 4.1692314 -3.3464477]], shape=(2, 2), dtype=float32)


In [20]:
import tensorflow as tf
predictions=tf.math.softmax(outputs.logits,axis=-1)# axis=-1 specifies the axis along which the softmax operation should be applied. In this case, -1 refers to the last axis of the logits tensor.
print(predictions)

tf.Tensor(
[[4.0195391e-02 9.5980465e-01]
 [9.9945587e-01 5.4418371e-04]], shape=(2, 2), dtype=float32)


In [28]:
l=model.config.id2label

In [31]:
import numpy
predictions=tf.math.softmax(outputs.logits,axis=-1).numpy()
for i,prob in enumerate(predictions):
  predicted_id=tf.argmax(prob).numpy()
  predicted_label=l[predicted_id]
  output_str = f"Sentence {i+1}: "
  for class_id, probs in enumerate(prob):
    label = l[class_id]
    output_str += f"{label}: {probs:.4f}, "

    # Remove trailing comma and space, and add conclusion
    output_str = output_str.rstrip(", ")
    output_str += f" | Predicted: {predicted_label}"

    print(output_str)

Sentence 1: NEGATIVE: 0.0402 | Predicted: POSITIVE
Sentence 1: NEGATIVE: 0.0402 | Predicted: POSITIVEPOSITIVE: 0.9598 | Predicted: POSITIVE
Sentence 2: NEGATIVE: 0.9995 | Predicted: NEGATIVE
Sentence 2: NEGATIVE: 0.9995 | Predicted: NEGATIVEPOSITIVE: 0.0005 | Predicted: NEGATIVE
